In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"

# Read Data

In [2]:
import numpy as np 
import pandas as pd 
for dirname, _, filenames in os.walk('../archive/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../archive/sqliv2.csv
../archive/sqli.csv
../archive/all.csv


In [3]:
df = pd.read_csv("/home/shuyu/others/cs315/data/all.csv")
train = pd.read_csv("/home/shuyu/others/cs315/data/train.csv")
val = pd.read_csv("/home/shuyu/others/cs315/data/val.csv")
test = pd.read_csv("/home/shuyu/others/cs315/data/test.csv")

In [4]:
import glob
import time
import pandas as pd

from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>
[nltk_data] Error loading stopwords: <urlopen error [Errno 111]
[nltk_data]     Connection refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>


# Pre-Process

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
temp = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
t1 = temp.fit_transform(df['Sentence'].values.astype('U')).toarray()
vocabulary = temp.get_feature_names()
vectorizer = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'), vocabulary=vocabulary)

In [7]:
train

,Sentence,Label
0,-2384'|| ( select 'vhcp' from dual where 1266 ...,1
1,hinkel,0
2,1 rlike ( select * from ( select ( sleep ( 5...,1
3,7081,0
4,It &apos;s apparent continuation Israel &apos...,0
...,...,...
26974,"Only people get old , says , smirking",0
26975,select * from users where id = 1 or \.<1 or 1...,1
26976,rali,0
26977,radley,0


In [8]:
X_train = vectorizer.fit_transform(train['Sentence'].values.astype('U')).toarray()
X_val = vectorizer.fit_transform(val['Sentence'].values.astype('U')).toarray()
X_test = vectorizer.fit_transform(test['Sentence'].values.astype('U')).toarray()

In [9]:
y_train = train['Label'].values
y_val = val['Label'].values
y_test = test['Label'].values

In [10]:
print('X_train: ', X_train.shape)
print('X_val: ', X_val.shape)
print('X_test: ', X_test.shape)


X_train:  (26979, 10319)
X_val:  (3372, 10319)
X_test:  (3374, 10319)


# Experiement

In [11]:
import joblib

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def performence(y_test, y_pred):
    a = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = \
        precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print("Accuracy : {:.4f}".format(a))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))
    print("F-Score : {:.4f}".format(fscore))

## Linear Regression

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)


In [15]:
y_pred=clf.predict(X_test)

In [16]:
y_test.shape

(3374,)

In [17]:
performence(y_test, y_pred)

Accuracy : 0.9600
Precision : 0.9622
Recall : 0.9600
F-Score : 0.9593


In [18]:
 joblib.dump(clf, '../model/LR.pkl')

['../model/LinearRegression.pkl']

## MLP

In [23]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [24]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


KeyboardInterrupt: 

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [ ]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)


In [ ]:
pred=model.predict(X_test)

In [ ]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [ ]:
performence(y_test, pred)

## SVM

In [19]:
from sklearn.svm import SVC

In [ ]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [ ]:
performence(y_test, y_pred)

In [ ]:
 joblib.dump(clf, '../model/SVM.pkl')

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

In [ ]:
performence(y_test, y_pred)

In [ ]:
joblib.dump(clf, '../model/KNN.pkl')

## Decision Tree

In [ ]:
from sklearn import tree

In [ ]:
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

In [ ]:
performence(y_test, y_pred)

In [ ]:
 joblib.dump(clf, '../model/DT.pkl')

In [ ]:
breakpoint()

## LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Bidirectional,LSTM
from keras.layers.core import Dense, Activation, Dropout

In [ ]:
X_input = np.expand_dims(X_train, 2)
y_input = np.expand_dims(y_train, 1)

In [ ]:
X_input.shape

In [ ]:
model = Sequential() #layers [1,50,50,50,50,1]

model.add(LSTM(input_shape=(None,1),units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(input_shape=(None,50),units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(input_shape=(None,50),units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(50,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1))
model.add(Activation("linear"))
# model.add(Activation("sigmoid"))


In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [ ]:
classifier_nn = model.fit(X_input,y_input,
                    epochs=10,
                    verbose=True,
#                     validation_data=(X_val, y_val),
                    batch_size=15)